In [1]:
!pip install -r ../requirements.txt 

ERROR: Could not open requirements file: [Errno 2] No such file or directory: '../requirements.txt'


In [2]:
import requests
import pandas as pd

In [3]:
def extract_products_by_category(category):
    url = f"https://de.openfoodfacts.org/cgi/search.pl?action=process&tagtype_0=categories&tag_contains_0=contains&tag_0={category}&page_size=100&json=true"
    response = requests.get(url)
    
    if response.status_code == 200:
        products_data = response.json()["products"]
        products_df = pd.DataFrame(products_data)
        return products_df
    else:
        print(f"Failed to fetch data for {category}")
        return None

categories = ["Drinks", "Dairy Products", "Beverages", "Nuts", "Sauces", "Sandwiches", "Snacks", "Meat alternatives", "Ready meals"]

combined_products = pd.DataFrame()

# Extract and combine products for each category
for category in categories:
    products_df = extract_products_by_category(category)
    if products_df is not None:
        combined_products = pd.concat([combined_products, products_df], ignore_index=True)

# Remove duplicates
combined_products.drop_duplicates(subset="product_name", keep="first", inplace=True)


OSError: Cannot save file into a non-existent directory: '..\data'

In [4]:
combined_file_name = "combined_products.csv"
combined_products.to_csv('./data/'+combined_file_name, index=False)
print(f"Combined products saved to {combined_file_name}")

Combined products saved to combined_products.csv


# Data Preprocessing

In [6]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [30]:
df = pd.read_csv("../data/combined_products.csv")

C:\Users\aligh\AppData\Local\Temp\ipykernel_31688\4183325667.py:1: DtypeWarning: Columns (76,83,119,129,213,240,258,301,303,305,306,308,309,311,312,313,314,317,321,323,324,325,326,328,333,334,335,340,341,342,343,344,349,356,359,361,363,364,365,366,367,374,377,388,390,391,392,393,402,403,404,405,406,407,411,415,418,421,430,431,433,436,437,438,439,440,441,442,443,444,445,446,447,448,449,451,452,453,454,455,456,457,458,459,460,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,505,506,508,509,510,511,512,513,519,520,521,522,523,526,527,528,529,530,531,532,534,535,536,537,538,539,540,541,542,543,545,546,551,552,553,554,555,556,557,558,559,560,561,562,567,568,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,621,626,633,636,637,638,642,643,644,645,646,647,648,651,

In [31]:
df.head(2)

,_id,_keywords,added_countries_tags,additives_debug_tags,additives_n,additives_old_n,additives_old_tags,additives_original_tags,additives_prev_original_tags,additives_tags,...,ingredients_text_de_ocr_1632393808_result,ingredients_text_de_ocr_1657142527,ingredients_text_de_ocr_1657142527_result,ingredients_text_fi_ocr_1590664929,ingredients_text_fi_ocr_1590664929_result,ingredients_text_sv_ocr_1554547178,ingredients_text_sv_ocr_1554547178_result,generic_name_sv_debug_tags,ingredients_text_sv_debug_tags,product_name_sv_debug_tags
0,3274080005003,"['boisson', 'cristaline', 'de', 'eau', 'eaux',...",[],[],0.0,0.0,[],[],[],[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5449000214911,"['aux', 'beverage', 'boisson', 'carbonated', '...",[],[],2.0,2.0,"['en:e150d', 'en:e338']","['en:e150d', 'en:e338']","['en:e150d', 'en:e338']","['en:e150d', 'en:e338']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df2 = df[["_id", "image_url", "brands", "pnns_groups_2", "nutriments", "nutriscore_data"]]

In [46]:
df2.head(10)

,_id,image_url,brands,pnns_groups_2,nutriments,nutriscore_data
0,3274080005003,https://images.openfoodfacts.org/images/produc...,Cristaline,Waters and flavored waters,"{'bicarbonate': 0.219, 'bicarbonate_100g': 0.0...","{'energy': 0, 'energy_points': 0, 'energy_valu..."
1,5449000214911,https://images.openfoodfacts.org/images/produc...,Coca cola,Sweetened beverages,"{'carbohydrates': 10.6, 'carbohydrates_100g': ...","{'energy': 180, 'energy_points': 6, 'energy_va..."
2,5449000214799,https://images.openfoodfacts.org/images/produc...,Coca-Cola,Artificially sweetened beverages,"{'carbohydrates': 0, 'carbohydrates_100g': 0, ...","{'energy': 1, 'energy_points': 1, 'energy_valu..."
3,5449000000996,https://images.openfoodfacts.org/images/produc...,"Coca-Cola HBC Polska Sp. z o.o.,Coca-Cola",Sweetened beverages,"{'carbohydrates': 10.6, 'carbohydrates_100g': ...","{'energy': 180, 'energy_points': 6, 'energy_va..."
4,3057640257773,https://images.openfoodfacts.org/images/produc...,Volvic,Waters and flavored waters,"{'bicarbonate': 0.074, 'bicarbonate_100g': 0.0...","{'energy': None, 'energy_points': 0, 'energy_v..."
5,5411188112709,https://images.openfoodfacts.org/images/produc...,Alpro,Plant-based milk substitutes,"{'calcium': 0.12, 'calcium_100g': 0.12, 'calci...","{'energy': 56, 'energy_points': 0, 'energy_val..."
6,5449000000439,https://images.openfoodfacts.org/images/produc...,Coca-Cola,Sweetened beverages,"{'carbohydrates': 11.2, 'carbohydrates_100g': ...","{'energy': 190, 'energy_points': 7, 'energy_va..."
7,5449000131836,https://images.openfoodfacts.org/images/produc...,COCA-COLA,Artificially sweetened beverages,"{'carbohydrates': 0, 'carbohydrates_100g': 0, ...","{'energy': 0.8, 'energy_points': 1, 'energy_va..."
8,7613035833272,https://images.openfoodfacts.org/images/produc...,perrier,Waters and flavored waters,"{'calcium': 0.16, 'calcium_serving': 0.16, 'ca...","{'energy': None, 'energy_points': 0, 'energy_v..."
9,3225350000501,https://images.openfoodfacts.org/images/produc...,"Pulco,Schweppes International Limited,Orangina...",Sweetened beverages,"{'alcohol_prepared_unit': '% vol', 'carbohydra...","{'energy': 14, 'energy_points': 1, 'energy_val..."


In [47]:
# drop columns that has no values
df2.dropna(subset=['nutriments'], inplace=True)

C:\Users\aligh\AppData\Local\Temp\ipykernel_31688\2612614600.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.dropna(subset=['nutriments'], inplace=True)


In [48]:
df2.head()

,_id,image_url,brands,pnns_groups_2,nutriments,nutriscore_data
0,3274080005003,https://images.openfoodfacts.org/images/produc...,Cristaline,Waters and flavored waters,"{'bicarbonate': 0.219, 'bicarbonate_100g': 0.0...","{'energy': 0, 'energy_points': 0, 'energy_valu..."
1,5449000214911,https://images.openfoodfacts.org/images/produc...,Coca cola,Sweetened beverages,"{'carbohydrates': 10.6, 'carbohydrates_100g': ...","{'energy': 180, 'energy_points': 6, 'energy_va..."
2,5449000214799,https://images.openfoodfacts.org/images/produc...,Coca-Cola,Artificially sweetened beverages,"{'carbohydrates': 0, 'carbohydrates_100g': 0, ...","{'energy': 1, 'energy_points': 1, 'energy_valu..."
3,5449000000996,https://images.openfoodfacts.org/images/produc...,"Coca-Cola HBC Polska Sp. z o.o.,Coca-Cola",Sweetened beverages,"{'carbohydrates': 10.6, 'carbohydrates_100g': ...","{'energy': 180, 'energy_points': 6, 'energy_va..."
4,3057640257773,https://images.openfoodfacts.org/images/produc...,Volvic,Waters and flavored waters,"{'bicarbonate': 0.074, 'bicarbonate_100g': 0.0...","{'energy': None, 'energy_points': 0, 'energy_v..."


# Flatten the column nutriments

In [49]:
from pandas import json_normalize
import json

In [50]:
# Assuming 'nutriments' is the column with JSON data in DataFrame df2
# Replace single quotes with double quotes in the 'nutriments' column
df2['nutriments'] = df2['nutriments'].str.replace("'", '"')

# Load JSON strings within 'nutriments' column
df2['nutriments'] = df2['nutriments'].apply(json.loads)

# Flatten the JSON column into separate columns
df_nutriments = pd.json_normalize(df2['nutriments'])

# Concatenate the flattened columns with the original DataFrame
df2 = pd.concat([df2, df_nutriments], axis=1)

# Drop the original 'nutriments' column if needed
df2.drop(columns=['nutriments'], inplace=True)

C:\Users\aligh\AppData\Local\Temp\ipykernel_31688\3015189922.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['nutriments'] = df2['nutriments'].str.replace("'", '"')
C:\Users\aligh\AppData\Local\Temp\ipykernel_31688\3015189922.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['nutriments'] = df2['nutriments'].apply(json.loads)


In [51]:
df2.head()

,_id,image_url,brands,pnns_groups_2,nutriscore_data,bicarbonate,bicarbonate_100g,bicarbonate_label,bicarbonate_serving,bicarbonate_unit,...,en-0_value,cholesterol_serving,alcohol_modifier,starch,starch_100g,starch_serving,starch_unit,starch_value,starch_label,proteins_label
0,3274080005003,https://images.openfoodfacts.org/images/produc...,Cristaline,Waters and flavored waters,"{'energy': 0, 'energy_points': 0, 'energy_valu...",0.219,0.0219,Bicarbonate,0.219,mg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5449000214911,https://images.openfoodfacts.org/images/produc...,Coca cola,Sweetened beverages,"{'energy': 180, 'energy_points': 6, 'energy_va...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5449000214799,https://images.openfoodfacts.org/images/produc...,Coca-Cola,Artificially sweetened beverages,"{'energy': 1, 'energy_points': 1, 'energy_valu...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5449000000996,https://images.openfoodfacts.org/images/produc...,"Coca-Cola HBC Polska Sp. z o.o.,Coca-Cola",Sweetened beverages,"{'energy': 180, 'energy_points': 6, 'energy_va...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3057640257773,https://images.openfoodfacts.org/images/produc...,Volvic,Waters and flavored waters,"{'energy': None, 'energy_points': 0, 'energy_v...",0.074,0.0074,NaN,0.074,mg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
# columns to keep in the final df
to_keep_columns = ["_id", "image_url", "brands", "pnns_groups_2", "energy-kj_100g", "sugars_100g",
                  "salt_100g", "saturated-fat_100g", "proteins_100g", "fiber_100g", 
                   "fruits-vegetables-nuts-estimate-from-ingredients_100g",
                  "sodium_100g", "nutrition-score-fr_100g"]

In [57]:
df_final = df2[to_keep_columns]

# randomly sample 150 items 

In [60]:
# Filter out rows where any column contains NaN values
df_no_na = df_final.dropna()

# Calculate the number of samples to take for each unique value in 'pnns_groups_2' column
num_samples = 150 // df_no_na['pnns_groups_2'].nunique()

# Create an empty DataFrame to store sampled data
sampled_data = pd.DataFrame()

# Iterate through each unique value in 'pnns_groups_2', sample the data, and append to the sampled_data DataFrame
for group_value in df_no_na['pnns_groups_2'].unique():
    group_data = df_no_na[df_no_na['pnns_groups_2'] == group_value].sample(min(num_samples, len(df_no_na[df_no_na['pnns_groups_2'] == group_value])))
    sampled_data = pd.concat([sampled_data, group_data])

# If the total number of samples is less than 150, sample the remaining randomly
remaining_samples = 150 - len(sampled_data)
if remaining_samples > 0:
    remaining_data = df_no_na.sample(min(remaining_samples, len(df_no_na)))
    sampled_data = pd.concat([sampled_data, remaining_data])

# Final sampled DataFrame with approximately 150 items
final_sample = sampled_data.sample(n=150)

In [62]:
final_sample.to_csv("./data/preprocessed_data.csv", index=False)